In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Word2Vec

<br>

>In this report we'll apply a pre-trained Word2Vec ML model to implement an algorithm to retrieve similar documents. Employed data and auxiliary scripts can be found at the report's [original repository](https://github.com/Benardi/bochica).

<br>

In [2]:
from functools import reduce
from ast import literal_eval
from math import log

import pandas as pd
from numpy import mean
from nltk import word_tokenize
from nltk.corpus import stopwords
from IPython.display import Markdown, display, HTML

default_stopwords = set(stopwords.words('portuguese'))

## Load Data

In [3]:
data = pd.read_csv("../output/results.csv")
data.head()

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


## Process text

In [27]:
def split_spread_words(corpus, delim):
    """ Split then spread alpha word with certain delimiters.

    Split words with alphabetical characters that have certain 
    delimiters then spread the resulting words across the corpus.

    :param list corpus: list of words.
    :param str delim: target delimiter.

    :return: updated list of words 

    :rtype: list
    """
    
    new_words = []
    for word in corpus:
        if any(c.isalnum() for c in word):
            new_words.extend(word.split(delim))
        else:
            new_words.append(word)

    return new_words


def word_processing(text):
    """ Process text and returns list of valid terms.

    Process text, removes stopwords, remove invalid chars 
    and returns list of valid terms.
    
    :param str text: text to be processed.

    :return: text as valid list of words 

    :rtype: list(str)
    """
    
    words = word_tokenize(text)
    
    # Put everything in lower case
    words = [word.lower() for word in words]
    
    # Remove words that don't have at least one alphabetical character 
    words = [word for word in words if any(c.isalnum() for c in word)]

    # Remove hyphen at end of word
    words = [word[:-1] if word[-1] == '-' else word for word in words]

    # Remove hyphen at beggining of word
    words = [word[1:] if word[0] == '-' else word for word in words]

    # Split words joined by en dash
    words = [word for line in words for word in line.split('–')] 
    words = [word for line in words for word in line.split('—')] # different encoding 

    # Split words joined by dot if they are alphabetical
    words = split_spread_words(words, '.')

    # Remove lone punctuation from the splits
    words = [word for word in words if any(c.isalnum() for c in word)]

    # Remove stopwords
    words = [word for word in words if word.lower() not in default_stopwords]
    
    return words

*  Let's process the news' titles to use them with our algorithm

In [28]:
data["sanitized_title"] = data["title"].apply(lambda text: word_processing(text))
data.head()

,title,subtitle,author,date,section,text,url,sanitized_title
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...,"[sociedade, rubens, paiva, facínoras, mataram]"
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...,"[justiça, suspende, decisão, proibia, forças, ..."
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...,"[governo, bolsonaro, prega, negacionismo, hist..."
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[pais, gabo, perceberam, filho, mentiroso]"
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[rádios, canadenses, banem, músicas, michael, ..."


## Apply transformation 'text to vector'

Let's load the pre-trained **Word2Vec** model from the gensim module.

This model has been trained with text in Portuguese taken from Wikipedia.

In [56]:
%%capture # Ignore gensim warnings 
from gensim.models import Word2Vec

model = Word2Vec.load('/home/benardi/workspace/bochica/data/pt/pt.bin')

In [38]:
def text_to_vectors(text, model):
    """ Transform text into a list of vectors.

    Applies a Word2Vec model to transfrom  
    already processed text into a list of vectors.
    
    :param list(str) text: list of terms.

    :param model obj: pre-trained Word2Vec model

    :return: list of vectors of same dimension 

    :rtype: list(list(number))
    """
    
    vectors = []
    for word in text:
        try:
            vectors.append(model.wv.get_vector(word))
        except:
            pass # Ignore unknown words

    return vectors

* Let's now apply the transformation to our processed titles

In [59]:
data["title_asvectors"] = data["sanitized_title"].apply(lambda title: text_to_vectors(title, model)) 
data.head()

,title,subtitle,author,date,section,text,url,sanitized_title,title_asvectors
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...,"[sociedade, rubens, paiva, facínoras, mataram]","[[0.20829841, -3.2993062, -1.9657193, 2.004485..."
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...,"[justiça, suspende, decisão, proibia, forças, ...","[[-0.31644273, 0.5114689, -0.5958715, 1.125206..."
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...,"[governo, bolsonaro, prega, negacionismo, hist...","[[-1.3232566, -1.6326014, -2.5689228, 2.607748..."
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[pais, gabo, perceberam, filho, mentiroso]","[[-0.712125, 0.67016095, -0.59830016, -0.76418..."
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[rádios, canadenses, banem, músicas, michael, ...","[[-1.2929834, -0.86154693, 1.2064326, 1.148315..."


## Find n most similar news

By making use of the metric *word mover distance* we can put together an algorithm to retrieve the n documents most similar to a given title.

In [54]:
from math import inf
from math import sqrt

def word_mover_distance(vctrs1, vctrs2):
    """ Calculates the word mover distance.

    Calculates the word mover distance on two
    vectors produced through Word2Vector.
    
    :param list(number) vctrs1: list of features as numbers.

    :param list(number) vctrs2: list of features as numbers.

    :return: distance between vectors 

    :rtype: number
    """    

    wmd = 0
    for vct1 in vctrs1:
        min_dist = inf
        for vct2 in vctrs2:
            min_dist = min(min_dist, sqrt(sum((vct1 - vct2) ** 2)))
  
        wmd += min_dist * 1.0/len(vctrs1)
 
    return wmd

def get_most_similar_news(news_title, news_df, model, n=3):
    """ Retrieve most similar documents/news.

    Applies word mover distance to retrieve n most similar
    documents/news to input news title.
    
    :param str news_title: title of the input news.

    :param pandas.core.frame.DataFrame news_df: df with news

    :param obj model: pre-trained Word2Vec model
    
    :param int n: number of documents to retrieve

    :return: n documents/news ranked by similarity 

    :rtype: pandas.core.frame.DataFrame
    """ 

    vectrs1 = text_to_vectors(news_title,model)
    temp = news_df.copy()
    temp["distance"] = temp["title_asvectors"]\
                       .apply(lambda vectrs2: word_mover_distance(vectrs1,vectrs2))

    result = temp.loc[lambda df: df.distance != 0]\ # It's pointless to retrieve the input news
             .nsmallest(n,'distance')\
             .drop(['sanitized_title', 'title_asvectors','distance'], axis=1)
    
    return result

### Is the implementation correct?

Let's use two distinct documents as input to check whether our implementation is correct

#### Document 1

In [118]:
doc_title1 = data.xs(4)["sanitized_title"]

pd.set_option('max_colwidth', 100)
pd.set_option('display.expand_frame_repr', False)

data.iloc[[4], :]

,title,subtitle,author,date,section,text,url,sanitized_title,title_asvectors
4,Rádios canadenses banem músicas de Michael Jackson da programação,Quebec Cogeco Media toma a decisão após queixas de ouvintes indignados com documentário sobre su...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e sucessos mundiais de pararam de tocar em algumas...,https://brasil.elpais.com/brasil/2019/03/06/cultura/1551902134_787786.html,"[rádios, canadenses, banem, músicas, michael, jackson, programação]","[[-1.2929834, -0.86154693, 1.2064326, 1.148315, -0.8980477, 0.10475973, -2.5438013, 1.9206856, -..."


Let's use the document of id=4 to test our implementation

In [87]:
get_most_similar_news(data.xs(4)["sanitized_title"], data, model, n=3)

,title,subtitle,author,date,section,text,url
179,“Nunca vou entender por que deixaram Michael J...,Diretor de ‘Deixando Neverland’ explica como l...,Tom C. Avendaño,24/03/2019 15:12:49,Cultura,O que o surpreendeu em relação à enorme reaçã...,https://brasil.elpais.com/brasil/2019/03/23/cu...
194,‘Tears in Heaven’: a tragédia que inspirou uma...,Março é uma época amarga para Eric Clapton. No...,Sara Navas,30/03/2019 14:24:02,Cultura,Quase uma década após a tragédia Lory del San...,https://brasil.elpais.com/brasil/2019/03/26/cu...
45,O trágico destino de Tom Ballard,Filho da alpinista Alison Hargreaves morta no...,Óscar Gogorza,03/03/2019 16:15:35,Esportes,Tom Ballard perdeu a mãe Alison Hargreaves a...,https://brasil.elpais.com/brasil/2019/03/02/de...


* The top result mentions Michael Jackson, so it's clearly satifactory
* The second result is less relevant, but it's still related to music
* The third result is much less relevant 

#### Document 2

In [101]:
doc_title2 = data.xs(245)["sanitized_title"]

data.iloc[[245], :]

,title,subtitle,author,date,section,text,url,sanitized_title,title_asvectors
245,Israel ataca Gaza e o Hamas lança foguetes ape...,Exército concentra tropas e limita as atividad...,Juan Carlos Sanz,26/03/2019 10:12:36,Internacional,O anúncio de cessar-fogo com mediação egípcia ...,https://brasil.elpais.com/brasil/2019/03/26/in...,"[israel, ataca, gaza, hamas, lança, foguetes, ...","[[0.28006357, -0.5752665, -1.1891643, 1.023996..."


In [119]:
get_most_similar_news(data.xs(245)["sanitized_title"], data, model, n=3)

,title,subtitle,author,date,section,text,url
110,Bolsonaro se despede de Israel com promessas econômicas e gafe sobre nazismo,Presidente encerrou visita a aliado Netanyahu no Museu do Holocausto e repetiu que nazismo é de ...,Lourdes Baeza,02/04/2019 16:03:54,Internacional,O presidente encerrou sua visita oficial a Israel com mais elogios ao Estado judaico ao qual r...,https://brasil.elpais.com/brasil/2019/04/02/internacional/1554216611_825972.html
100,O que há por trás do idílio político de Bolsonaro com Israel e Netanyahu,Presidente do Brasil chega neste domingo a Jerusalém sem esclarecer totalmente a incógnita sobre...,Naiara Galarraga Gortázar,30/03/2019 16:21:31,Internacional,O primeiro-ministro israelense foi o convidado mais bem-quisto do presidente do Brasil em sua po...,https://brasil.elpais.com/brasil/2019/03/30/internacional/1553969475_732931.html
171,A fuga simbólica de Bolsonaro para Israel no 55º aniversário da ditadura brasileira,Presidente deixou o país na data que defendeu celebrar e brasileiros fiéis à democracia deixara...,Juan Arias,01/04/2019 08:26:07,Opinião,O presidente brasileiro está se acostumando a atirar a pedra e esconder a mão. Desta vez ten...,https://brasil.elpais.com/brasil/2019/04/01/opinion/1554078976_839038.html


* All three retrieved news are closely related to the input document as they all mentioned Israel.

> There's evidence to support that the implementation is correct. The evidence is especially clear in view of the second document.